[Inspirado en este video](https://www.youtube.com/watch?v=p2sTJYoIwj0)

In [10]:
!git clone https://github.com/Zelechos/Generador-Css.git

Cloning into 'Generador-Css'...
remote: Enumerating objects: 335, done.
remote: Counting objects: 100% (335/335), done.
remote: Compressing objects: 100% (271/271), done.
remote: Total 335 (delta 153), reused 172 (delta 56), pack-reused 0
Receiving objects: 100% (335/335), 3.47 MiB | 25.91 MiB/s, done.
Resolving deltas: 100% (153/153), done.


# Instalamos el modelo preentrenado para hacer pruebas

In [11]:
!pip install keras-transformer

  Created wheel for keras-transformer: filename=keras_transformer-0.40.0-py3-none-any.whl size=12305 sha256=53676122351456b82fc5dbb403f1a829d4aa9bb73f296f0c9eba8b6108591335
  Stored in directory: /root/.cache/pip/wheels/46/68/26/692ed21edd832833c3b0a0e21615bcacd99ca458b3f9ed571f
  Created wheel for keras-embed-sim: filename=keras_embed_sim-0.10.0-py3-none-any.whl size=3960 sha256=7fc9250cacb74c26e2df46bd149f4d7c047b053b78f9c6c46fcb2ff6a9927bf5
  Stored in directory: /root/.cache/pip/wheels/81/67/b5/d847588d075895281e1cf5590f819bd4cf076a554872268bd5
  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.16.0-py3-none-any.whl size=4668 sha256=462d2dcdaa9ad2f24f785205153c4253194b799f78fc93e5a0681ab23c826cc3
  Stored in directory: /root/.cache/pip/wheels/85/5d/1c/2e619f594f69fbcf8bc20943b27d414871c409be053994813e
  Created wheel for keras-multi-head: filename=keras_multi_head-0.29.0-py3-none-any.whl size=14993 sha256=884d8134c56669cc5645f7b1aa42804040163cbdfa72

In [12]:
import numpy as np
from keras_transformer import get_model, decode
np.random.seed(0)

## Iniciamos con el preprocesamiento de los datos

In [14]:
import json

with open('/content/Generador-Css/dataset/TEST.json') as file:
    data = json.load(file)

#accedemos a la los datos
data_styles = data["styles"]
print(data_styles[0])


#accediendo a los estilos
styles = data_styles[0]["style"]
print("\nstyles access => ",styles)


#accediendo a los selectores
selector = data_styles[0]["selector"]
print("\nselector access => ",selector)

#cantidad de datos
print("cantidad de estilos en el dataset : ", len(data_styles))

{'style': 'body { overflow:hidden ; background:rgb(25,35,125) } ', 'selector': 'body'}

styles access =>  body { overflow:hidden ; background:rgb(25,35,125) } 

selector access =>  body
cantidad de estilos en el dataset :  1337


In [15]:
data_css = []
styles = []
selectors = []
for data in data_styles:
  selectors.append(data["selector"])
  styles.append(data["style"])
 
# print(selectors)
# print(styles)
data_css.append(selectors)
data_css.append(styles)
print(data_css[0])
print(data_css[1])

['body', 'div.drop-container', 'div.drop-container', 'div.drop-container', 'div.drop-container', 'div.drop-container', 'div.drop', 'h1', '@font-face', 'body', '.logo', '.switch-left', '.pad-left', '.switch-right', '.pad-right', '.line', '.text', '.text', '.text', '.text', '.text', '.text', '.text', '.text', '.text', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', '.text', '.reload', '.reload', '.reload', '.reload', '.reload', '.reload', '.reload', '.reload', '.reload', '.reload', '.reload', '.reload', '.logo', 'html,body', 'body', '*', '*', '*', '.css_lab', '.witch', 'body', 'body img', 'img', 'body h1', 'h1', 'body a', 'a', '@keyframes glow', 'from', 'to', 'body', 'h1', 'ul', 'li', 'li:hover span', 'li.invisible', 'li.animate', 'span', 'span:before', 'html', 'body', 'body', 'body', 'body', 'body'

In [16]:
print(selectors)
print(data_css[1][0])

['body', 'div.drop-container', 'div.drop-container', 'div.drop-container', 'div.drop-container', 'div.drop-container', 'div.drop', 'h1', '@font-face', 'body', '.logo', '.switch-left', '.pad-left', '.switch-right', '.pad-right', '.line', '.text', '.text', '.text', '.text', '.text', '.text', '.text', '.text', '.text', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', 'body', '.text', '.reload', '.reload', '.reload', '.reload', '.reload', '.reload', '.reload', '.reload', '.reload', '.reload', '.reload', '.reload', '.logo', 'html,body', 'body', '*', '*', '*', '.css_lab', '.witch', 'body', 'body img', 'img', 'body h1', 'h1', 'body a', 'a', '@keyframes glow', 'from', 'to', 'body', 'h1', 'ul', 'li', 'li:hover span', 'li.invisible', 'li.animate', 'span', 'span:before', 'html', 'body', 'body', 'body', 'body', 'body'

Creaemos los tokens para el manejo de textos

In [17]:
selector_tokens = []
for sentence in selectors:
  selector_tokens.append(sentence.split(' '))

print(selector_tokens[0])

style_tokens = []
for sentence in styles:
  style_tokens.append(sentence.split(' '))

print(style_tokens[89])

['body']
['li', '{', 'list-style:none', ';', 'margin:.5em', '0', '0', ';', 'font-size:1.75em', '}', '']


Creamos una funcion para construir finalmente los tokens

In [18]:
def build_token_dict(token_list):
  token_dict = {
      '<PAD>': 0,
      '<START>': 1,
      '<END>': 2
  }

  # aqui leemos el token creado
  for tokens in token_list:
    for token in tokens:
      if token not in token_dict:
        token_dict[token] = len(token_dict)
  
  return token_dict

Creamos los tokens usando nuestra funcion anterior

In [19]:
# estos son los tokens que le pasaremos al encoder que seran los selectores
selector_token_dict = build_token_dict(selector_tokens)

# estos son los tokens que le pasaremos al decoder que seran los styles
style_token_dict = build_token_dict(style_tokens)

# por ultimo invertimos nuestro diccionario ya que nos tiene que devolver texto no numeros
style_token_dict_inv = {v:k for k,v in style_token_dict.items()}

# Imprimimos nuestros diccinarios
print(selector_token_dict)
print(style_token_dict)
print(style_token_dict_inv)

{'<PAD>': 0, '<START>': 1, '<END>': 2, 'body': 3, 'div.drop-container': 4, 'div.drop': 5, 'h1': 6, '@font-face': 7, '.logo': 8, '.switch-left': 9, '.pad-left': 10, '.switch-right': 11, '.pad-right': 12, '.line': 13, '.text': 14, '.reload': 15, 'html,body': 16, '*': 17, '.css_lab': 18, '.witch': 19, 'img': 20, 'a': 21, '@keyframes': 22, 'glow': 23, 'from': 24, 'to': 25, 'ul': 26, 'li': 27, 'li:hover': 28, 'span': 29, 'li.invisible': 30, 'li.animate': 31, 'span:before': 32, 'html': 33, 'header': 34, 'article': 35, 'h3,section': 36, 'h3': 37, 'section': 38, 'h3:before': 39, 'h3:before,section:before': 40, 'section:before': 41, 'p': 42, 'body,html': 43, 'div.codepen': 44, 'div': 45, '.codepen': 46, 'aside.context': 47, '.context': 48, 'aside': 49, 'a:hover': 50, 'footer': 51, '.img-centered': 52, '.bg-light-gray': 53, '.bg-darkest-gray': 54, '.btn-primary': 55, '.navbar-default': 56, '.navbar-default.navbar-brand': 57, '.navbar-default.navbar-collapse': 58, '.navbar-default.nav': 59, '.nav

Agregamos cada token START , PAD y END a cada frase de nuestro datos para tener longitud igual para no causar errores de tamaño en numpy(dado que en numpy todos lo vectores deben ser de la misma longitud)

In [20]:
encoder_tokens = [['<START>'] + tokens + ['<END>'] for tokens in selector_tokens]
decoder_tokens = [['<START>'] + tokens + ['<END>'] for tokens in style_tokens]
output_tokens = [ tokens + ['<END>'] for tokens in style_tokens]

# Vamos a buscar el elemento con mayor longitud para darle la misma longitud a todos los elementos y evitar el error de longitud en numpy
selector_max_len = max(map(len, encoder_tokens))
styles_max_len = max(map(len, decoder_tokens))

# aqui agregamos el token <PAD> a los valores directamente
encoder_tokens = [ tokens + ['<PAD>'] * (selector_max_len - len(tokens)) for tokens in encoder_tokens]
decoder_tokens = [ tokens + ['<PAD>'] * (styles_max_len - len(tokens)) for tokens in decoder_tokens]
output_tokens = [ tokens + ['<PAD>'] * (styles_max_len - len(tokens)) for tokens in output_tokens]

ahora podemos ver que esta tokenizado nuestros elementos

In [21]:
print(encoder_tokens[0])
print(decoder_tokens[0])
print(output_tokens[0])

['<START>', 'body', '<END>', '<PAD>', '<PAD>']
['<START>', 'body', '{', 'overflow:hidden', ';', 'background:rgb(25,35,125)', '}', '', '<END>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<

## Importante!!! ahora tenemos que representar numericamente los valores para crear nuestro vector de numpy tambien podemos crear tensores de Pytorch partiendo de esta logica TOMAR EN CUENTA!!!!!!

In [22]:
encoder_input = [list(map(lambda x: selector_token_dict[x], tokens)) for tokens in encoder_tokens]
decoder_input = [list(map(lambda x: style_token_dict[x], tokens)) for tokens in decoder_tokens]
output_decoded = [list(map(lambda x: [style_token_dict[x]], tokens)) for tokens in output_tokens]

ahora si lo imprimimos nos dara todos los elementos en valores numericos

In [23]:
print(encoder_input[0])
print(decoder_input[0])
print(output_decoded[0])

[1, 3, 2, 0, 0]
[1, 3, 4, 5, 6, 7, 8, 9, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[3], [4], [5], [6], [7], [8], [9], [2], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], 

ahora si traemos nuestro modelo pre entrenado

In [24]:
model = get_model(
    token_num = max(len(selector_token_dict), len(style_token_dict)),
    embed_dim = 32,
    encoder_num = 3,
    decoder_num = 3,
    head_num = 4,
    hidden_dim = 128,
    dropout_rate = 0.05,
    use_same_embed = False
)

model.compile('adam', 'sparse_categorical_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Encoder-Input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 Encoder-Token-Embedding (Embed  [(None, None, 32),  121984      ['Encoder-Input[0][0]']          
 dingRet)                        (3812, 32)]                                                      
                                                                                                  
 Encoder-Embedding (TrigPosEmbe  (None, None, 32)    0           ['Encoder-Token-Embedding[0][0]']
 dding)                                                                                           
                                                                                              

realizamos el entrenamiento

In [34]:
x = [np.array(encoder_input), np.array(decoder_input)]
y = np.array(output_decoded)

model.fit(x, y, epochs=5000, batch_size = 128)

Se han truncado las últimas 5000 líneas del flujo de salida.
11/11 [==============================] - 1s 95ms/step - loss: 0.0210
Epoch 2502/5000
11/11 [==============================] - 1s 94ms/step - loss: 0.0209
Epoch 2503/5000
11/11 [==============================] - 1s 95ms/step - loss: 0.0210
Epoch 2504/5000
11/11 [==============================] - 1s 97ms/step - loss: 0.0212
Epoch 2505/5000
11/11 [==============================] - 1s 98ms/step - loss: 0.0211
Epoch 2506/5000
11/11 [==============================] - 1s 97ms/step - loss: 0.0213
Epoch 2507/5000
11/11 [==============================] - 1s 95ms/step - loss: 0.0211
Epoch 2508/5000
11/11 [==============================] - 1s 96ms/step - loss: 0.0210
Epoch 2509/5000
11/11 [==============================] - 1s 97ms/step - loss: 0.0210
Epoch 2510/5000
11/11 [==============================] - 1s 94ms/step - loss: 0.0209
Epoch 2511/5000
11/11 [==============================] - 1s 93ms/step - loss: 0.0210
Epoch 2512/5000
11/1

In [35]:
def GeneratorCSS(sentences):
  sentence_tokens = [tokens + ['<END>', '<PAD>'] for tokens in [sentences.split(' ')]]
  tr_input = [list(map(lambda x: selector_token_dict[x], tokens)) for tokens in sentence_tokens][0]
  decoded = decode(
      model,
      tr_input,
      start_token = style_token_dict['<START>'],   
      end_token = style_token_dict['<END>'],   
      pad_token = style_token_dict['<PAD>']   
  )

  #aqui imprimimos los resultados
  print("Selector : {} ".format(sentences))
  print("Style : {}".format(' '.join(map(lambda x: style_token_dict_inv[x], decoded[1:-1]))))


In [40]:
GeneratorCSS('.image-mosaic')

Selector : .image-mosaic 
Style : .image-mosaic { display:grid ; gap:1rem ; grid-template-columns:repeat(auto-fit,minmax(240px,1fr)) ; grid-auto-rows:240px ; } .card { display:flex ; flex-direction:column ; justify-content:center ; align-items:center ; background:#DD068A ; font-size:3rem ; color:#1517D7 ; box-shadow:rgba(3,8,20,.1) 0 .15rem .5rem,rgba(2,8,20,.1) 0 .075rem .175rem ; height:100% ; width:100% ; border-radius:4px ; transition:all 500ms ; overflow:hidden ; background-size:cover ; background-position:center ; background-repeat:no-repeat ; padding:0 ; margin:0 } @media screen and (min-width:600px) { .card-tall { grid-row:span 2/auto ; } .card-wide { grid-column:span 2/auto ; } } 


Guardando el modelo!!!!

In [41]:
model.save("generador_css_1.h5")

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [42]:
!ls

Generador-Css  generador_css_1.h5  sample_data


Convertimos con una libreria de tensorflow a js

In [43]:
!pip install tensorflowjs

     |████████████████████████████████| 77 kB 3.5 MB/s 


creamas una carpeta en donde se pondran nuestro modelo convertido en js dado que son muchos archivos

In [44]:
!mkdir modelo_js

realizamos la exportacion de la carpeta

In [45]:
!tensorflowjs_converter  --input_format keras generador_css_1.h5 modelo_js

 confirmamos si se generaron los archivos (bin,json)

In [46]:
!ls modelo_js

group1-shard1of1.bin  model.json
